## パイプライン

### パイプラインとは
#### 変換器(特徴量の生成と選択)と予測気を直列に繋げ，一連の処理をまとめて実行できるようにする仕組みのことをパイプラインと言う．
### メリット
1. いくつかの前処理を実行している場合でも，fit や predict をパイプラインに対して1度だけ呼ぶだけで良い
2. パイプラインに対して，グリッドサーチを行う際，パラメータを1度に指定できる．fit を読んだ後の変換器をキャッシュでき，性能向上が期待できる．
3. transformer と estimatorに同じデータが使われるため，交差検証の際にデータがリークされることを防げる

### 実装

1. GridSearchCV
2. クロスバリデーションをしながらパラメータチューニング
3. 評価指標の平均値を計算
4. パイプライン構築
    - グリッドサーチによる出力されたハイパーパラメータを使う
    - 前処理は，標準化，主成分分析による次元削除
    - 推定器には，重回帰，ランダムフォレストを使用
    - クロスバリデーションをしながら学習

In [58]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
import pickle
from sklearn.model_selection import train_test_split, KFold, cross_val_score, StratifiedKFold

# パラメーター調整
from sklearn.model_selection import GridSearchCV

# 変換器
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# 予測器
from sklearn.ensemble import ExtraTreesClassifier ,  GradientBoostingClassifier, RandomForestClassifier


# パイプライン
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# 評価関数
from sklearn.metrics import recall_score,precision_score,accuracy_score,confusion_matrix, roc_curve,roc_auc_score,auc

# パイプラインの過程
from sklearn import set_config

### データの読み込み

In [20]:
df = pd.read_csv('../Data-science//data//breast_cancer_imbalance.csv')

df.head()

In [21]:
skf = StratifiedKFold(n_splits = 5, shuffle = True)
df["fold"] = -1
for fold, (train_idx, test_idx) in enumerate(skf.split(X = df, y = df["target"])):
    df.loc[test_idx, "fold"] = fold
print(df["fold"].value_counts())

fold = 0
train = df.loc[df["fold"] != fold].copy()
test = df.loc[df["fold"] == fold].copy()
print(train.shape, test.shape)

feat_cols = train.drop(columns = ["fold", "target"]).columns.tolist()
print(feat_cols)

X_train = train[feat_cols]
X_test = test[feat_cols]
y_train = train["target"]
y_test = test["target"]

0    80
1    80
2    80
3    80
4    80
Name: fold, dtype: int64
(320, 32) (80, 32)
['mean radius', 'mean texture', 'mean perimeter', 'mean area', 'mean smoothness', 'mean compactness', 'mean concavity', 'mean concave points', 'mean symmetry', 'mean fractal dimension', 'radius error', 'texture error', 'perimeter error', 'area error', 'smoothness error', 'compactness error', 'concavity error', 'concave points error', 'symmetry error', 'fractal dimension error', 'worst radius', 'worst texture', 'worst perimeter', 'worst area', 'worst smoothness', 'worst compactness', 'worst concavity', 'worst concave points', 'worst symmetry', 'worst fractal dimension']


### 前処理

#### 特徴量選択

In [22]:
features = X_train.columns

#### 標準化

In [9]:
scaler = StandardScaler()

#### 次元削除

次元削減にはPCAを使う．

In [23]:
pca = PCA()

### パイプライン構築

In [24]:
# 変換器パイプライン(前処理パイプライン)
## 前処理を行う
transformer = Pipeline(steps=[
                              ('StandScaler', scaler),
                              ('pca', pca),
])

# 特徴量 -> 変換器
preprocessor = ColumnTransformer(
    transformers=[('transform', transformer, features)]
)


In [25]:
# 推定器の定義

gbc = GradientBoostingClassifier(
  random_state=53,
  min_samples_split = 5, 
  min_samples_leaf = 50,
  max_depth = 5,
  subsample = 0.8
)

etc = ExtraTreesClassifier(
    random_state=53,
    min_samples_split = 5,
    min_samples_leaf = 50,
    max_depth = 5,
    n_jobs=-1
)

rfc = RandomForestClassifier(
    max_depth=None,
    max_features='auto',
    max_leaf_nodes=None,
    min_samples_leaf=1,
    min_samples_split=2,
    n_jobs=-1,
    random_state=53,
    verbose=0
)

In [26]:
## パイプライン全体：変換器パイプライン‐＞予測器（推定器）
pipe = Pipeline(
    steps=[
           ('preprocessor', preprocessor),
           ('classifier', gbc)
    ]
)

#### パラメータチューニング

グリッドサーチを行い，パラメータチューニングを行う

Scikit-Learnオブジェクトと GridSearchCVを組み合わせる場合，パラメータ名のみで良いが，Pipelineには複数のオブジェクトが含まれるため，それらを識別するための名前を追加する必要がある．

パラメータ名の前に，パイプラインを定義した際に推定器の’key+__’を付ける．今回の場合はregressor．

In [27]:
# パラメータ
## 推定器は3つ

params_grid = [{'classifier':[gbc],
                'classifier__n_estimators': list(range(20, 101, 10)),
                'classifier__max_depth': [2, 3, 4, 6],
                'classifier__learning_rate': list(np.arange(0.05, 0.20, 0.01))},
               {'classifier': [etc],
                'classifier__n_estimators': [50, 75, 100, 125, 150],
                'classifier__max_depth': [2, 3, 4, 6]},    
               {'classifier': [rfc],
                'classifier__n_estimators': [50, 75, 100, 125, 150],
                'classifier__max_depth': [2, 3, 4, 6]}
]

gs = GridSearchCV(
    estimator=pipe,
    param_grid=params_grid,
    scoring='recall', # 'precision', 'roc_auc'
    cv=7,
    n_jobs = -1,
    refit=True,
    verbose=10
)


In [28]:
gs.fit(X_train, y_train)

Fitting 7 folds for each of 616 candidates, totalling 4312 fits


GridSearchCV(cv=7,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('transform',
                                                                         Pipeline(steps=[('StandScaler',
                                                                                          StandardScaler()),
                                                                                         ('pca',
                                                                                          PCA())]),
                                                                         Index(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius err...
                         {'classifier': [ExtraTreesClassifier(max_depth=5,
                                                    

#### cv_results_ で列ヘッダーとしてのキーと列としての値を持つdictを呼び出せる．これは DataFrame インポートできる．今回は最も精度が良かったインデックスを表示．

In [29]:
cv_result = pd.DataFrame(gs.cv_results_)
cv_result.loc[gs.best_index_]

mean_fit_time                                                               0.059888
std_fit_time                                                                0.010917
mean_score_time                                                             0.007313
std_score_time                                                              0.001862
param_classifier                   GradientBoostingClassifier(learning_rate=0.05,...
param_classifier__learning_rate                                                 0.05
param_classifier__max_depth                                                        2
param_classifier__n_estimators                                                    20
params                             {'classifier': GradientBoostingClassifier(lear...
split0_test_score                                                                1.0
split1_test_score                                                                1.0
split2_test_score                                                

In [30]:
# 最も精度が高かった組合せのPipelineの過程
gs.best_estimator_

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('transform',
                                                  Pipeline(steps=[('StandScaler',
                                                                   StandardScaler()),
                                                                  ('pca',
                                                                   PCA())]),
                                                  Index(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimete...
       'worst radius', 'worst texture', 'worst perimeter', 'worst area',
       'worst smoothness', 'worst compactness', 'worst concavity',
       'worst concave points', 'worst symmetry', 'worst fractal dimension'],
      dtype='object'))])),
                ('classifier',
 

In [43]:
 best_params = gs.best_params_
 best_params

{'classifier': GradientBoostingClassifier(learning_rate=0.05, max_depth=2, min_samples_leaf=50,
                            min_samples_split=5, n_estimators=20,
                            random_state=53, subsample=0.8),
 'classifier__learning_rate': 0.05,
 'classifier__max_depth': 2,
 'classifier__n_estimators': 20}

In [44]:
gs.best_score_

1.0

In [45]:
gs.score(X_test, y_test)

1.0

In [46]:
pred = gs.predict(X_test)
pred

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [51]:
accuracy = accuracy_score(y_test, pred)

print(accuracy)

0.8875


In [52]:
confusion_matrix2 = confusion_matrix(y_test, pred)
print(confusion_matrix2)

recall = recall_score(y_test, pred)
precision = precision_score(y_test, pred)
print(recall)
print(precision)

[[ 0  9]
 [ 0 71]]
1.0
0.8875


In [63]:
fpr, tpr, thresholds = (roc_curve(y_test, pred))
print(roc_curve(y_test, pred))

print(auc(fpr, tpr))

(array([0., 1.]), array([0., 1.]), array([2, 1]))
0.5


In [57]:
print(roc_auc_score(y_test, pred))

0.5
